In [1]:
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")
using PyCall
using Distributions
unshift!(PyVector(pyimport("sys")["path"]), "")
unshift!(PyVector(pyimport("sys")["path"]), "..")
unshift!(PyVector(pyimport("sys")["path"]), "../Ad05RTheta1RSteer05Abs0FR200")
unshift!(PyVector(pyimport("sys")["path"]), "../CommonFiles")
@pyimport python2juliaReuse as p2j

In [2]:
# policies = ["Baseline/Rewards/b_15e3_3","Retrain/Rewards/rarl","Retrain/Rewards/rarlP10",
#             "FSP/Rewards/nBnoTR","FSP/Rewards/nBnoTRP10"]

In [3]:
pc = p2j.PolicyContainer()
path = "Data/Sep1/Ad05RTheta1RSteer05Abs0FR200/"
# policies = readdir(string(path,"Policy"))
# pnames = []
# for i = 1:length(policies)
#     if policies[i] != ".DS_Store"
#         push!(pnames,policies[i][1:length(policies[i])-4])
#     end
# end
# println(pnames)
pnames = ["b_15e3","rarl_5e3","fsp_4e3"]
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")


2018-09-07 15:38:35.240744: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-09-07 15:38:35.240759: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-09-07 15:38:35.240763: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-09-07 15:38:35.240766: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.
07-Sep 15:38:35:WARNING:root:replacing docs for 'close :: Tuple{ZMQTransport}' in module 'Main'.
07-Sep 15:38:35

run_env_server_rarl (generic function with 1 method)

In [4]:
mutable struct Counter
    i::Int
    I::Int
    num_collision::Int
    rate_collision::Float64
    total_reward::Float64
    single_reward::Float64
    avg_reward::Float64
    max_reward::Float64
    min_reward::Float64
    rewards::Array{Float64}
    done::Bool
    direction::Int
    total_speed::Float64
    single_speed::Float64
    avg_speed::Float64
    speeds::Array{Float64}
    single_time::Float64
    total_time::Float64
    time_between_collisions::Float64
    total_reward_d::Float64
    single_reward_d::Float64
    avg_reward_d::Float64
    rewards_d::Array{Float64}
end
function Counter(N::Int)
    return Counter(0,N,0,0.0,0.0,0.0,0.0,-Inf,Inf,zeros(N),false,-1,
        0.0,0.0,0.0,zeros(N),0.0,0.0,0.0,0.0,0.0,0.0,zeros(N))
end

Counter

In [5]:
function handle_action!(pc,scene,models,roadway,counter,rarl)
    state = get_observation(scene,models,roadway)
    if rarl
#         action1_arr = pc[:getAction_just](state)
        dist = pc[:getActionDistribution_just](state)
        action1_arr = dist["mean"]
    else
        action1_arr = pc[:getAction_just_baseline](state)
    end
    action1_arr = tanh.(action1_arr)

    dacc = 0.0
    dsteer = 0.0
    action = Egoaction(action1_arr[1],action1_arr[2],dacc,dsteer)
    
    done, bump = simulate_action!(action,scene,models,roadway)
    counter.single_time += TIMESTEP
    counter.total_time += TIMESTEP
    reward = reward_fn(action,done,scene,models,roadway)
    if bump
        counter.num_collision += 1
    else
        counter.total_reward_d += reward
        counter.single_reward_d += reward
    end
    
    counter.total_reward += reward
    counter.single_reward += reward
    
    speed = scene[1].state.v
    counter.single_speed += speed
    counter.total_speed += speed
    
    if done
        pc[:resetPolicy](1)
        rand_ego!(scene,models,roadway)
        
        counter.i += 1
        if counter.single_reward > counter.max_reward
            counter.max_reward = counter.single_reward
        end
        if counter.single_reward < counter.min_reward
            counter.min_reward = counter.single_reward
        end
        if counter.i > counter.I
            counter.done = true
            counter.avg_reward = counter.total_reward/counter.I
            counter.rate_collision = counter.num_collision/counter.I
            counter.avg_reward_d = counter.total_reward_d/counter.I
            counter.avg_speed = counter.total_speed/(counter.total_time/TIMESTEP)
            counter.time_between_collisions = counter.total_time/counter.num_collision
        else
            counter.rewards[counter.i] = counter.single_reward
            counter.rewards_d[counter.i] = counter.single_reward_d
            counter.speeds[counter.i] = counter.single_speed/(counter.single_time/TIMESTEP)
        end
        counter.single_reward = 0.0
        counter.single_reward_d = 0.0
        counter.single_speed = 0.0
        counter.single_time = 0.0
    end
    
    
end

handle_action! (generic function with 1 method)

In [7]:
alphas = 1.0:0.2:4.0
log_path = string(path,"AxisLengthNoNoise/")
scene, models, roadway = initialize_env()
for alpha in alphas
    println("alpha: ",alpha)
    for pname in pnames
        println(pname)
        policy_path = string(path,"Policy/",pname,".pkl")
        pc[:reset_policy](policy_path=policy_path)
        N = 200

        counter = Counter(N)
        car_a = alpha*CAR_A/AXLE_DISTANCE
        car_b = alpha*CAR_B/AXLE_DISTANCE
        rand_ego!(scene,models,roadway,car_a=car_a,car_b=car_b)
        pc[:resetPolicy](1)
        while !counter.done
            handle_action!(pc,scene,models,roadway,counter,true)
        end
        println(counter.avg_reward)
        println(counter.rate_collision)

        writedlm(string(log_path,pname,"_rewards_axis",alpha,".txt"),counter.rewards)
        writedlm(string(log_path,pname,"_rewards_d_axis",alpha,".txt"),counter.rewards_d)
        writedlm(string(log_path,pname,"_speeds_axis",alpha,".txt"),counter.speeds)
        writedlm(string(log_path,pname,"_collision_rate_axis",alpha,".txt"),counter.rate_collision)
#         writedlm(string(log_path,pname,"_time_between_collisions_pareto",alpha,".txt"),(counter.total_time+counter2.total_time)/(counter.num_collision+counter2.num_collision))
    end
end

alpha: 1.0
b_15e3
815.6882905367378
0.0
rarl_5e3
507.0799743214086
0.0
fsp_4e3
749.320434656084
0.0
alpha: 1.2
b_15e3
815.624636325939
0.0
rarl_5e3
507.7291179805364
0.0
fsp_4e3
750.0275860559168
0.0
alpha: 1.4
b_15e3
816.3637297487278
0.0
rarl_5e3
508.04312604334393
0.0
fsp_4e3
751.0081450080854
0.0
alpha: 1.6
b_15e3
816.2017471241936
0.0
rarl_5e3
507.3377624035582
0.0
fsp_4e3
749.9058223629556
0.0
alpha: 1.8
b_15e3
816.4207492754805
0.0
rarl_5e3
507.5945114522641
0.0
fsp_4e3
751.8021735116331
0.0
alpha: 2.0
b_15e3
815.9987986364853
0.0
rarl_5e3
507.62631041336476
0.0
fsp_4e3
750.846694590972
0.0
alpha: 2.2
b_15e3
817.0660755116653
0.0
rarl_5e3
507.2201992366049
0.0
fsp_4e3
750.5023041087782
0.0
alpha: 2.4
b_15e3
814.2035001590802
0.0
rarl_5e3
507.15985002031033
0.0
fsp_4e3
750.98064840056
0.0
alpha: 2.6
b_15e3
816.3793112252431
0.0
rarl_5e3
506.85507415904345
0.0
fsp_4e3
752.227624294127
0.0
alpha: 2.8
b_15e3
816.2994666008691
0.0
rarl_5e3
507.49091051642165
0.0
fsp_4e3
749.116750988